In [127]:
import numpy as np
import pandas as pd

class Instancia():
    def __init__(self,csvfile):
        self._equipos=0
        self._partidos=0
        self._cmm = None
        self.mass=None
        self.diferencia=None
        self.ganados = None
        self.totales = None
        self.ratings=None
        f = open(csvfile, "r")
        i=0
        for line in list(f):
            if i == 0: # header
                i+=1
                columns = line.split()
                self._equipos = int(columns[0])
                self._partidos = int(columns[1])
                self._cmm = np.empty([self._equipos,self._equipos])
                self.ganados= np.empty([self._equipos,self._equipos])
                self.totales = np.zeros(self._equipos)
                self.diferencias = np.zeros(self._equipos)
                self.ratings = np.ones(self._equipos)*100
                for i in range(0,self._equipos):
                    for j in range(0,self._equipos):
                        self.ganados[i,j]=0.0
            else:
                data = line.split();
                print(data)
#                     set_trace()
                equipo1=int(data[1])-1
                goles1=int(data[2])-1
                equipo2=int(data[3])-1
                goles2=int(data[4])-1
                self.totales[int(data[1])-1]+=1
                self.totales[int(data[3])-1]+=1
                self.diferencias[int(data[1])-1]+=int(data[2])

                self.diferencias[int(data[1])-1]-=int(data[4])


                self.diferencias[int(data[3])-1]+=int(data[4])

                self.diferencias[int(data[3])-1]-=int(data[2])
                
                r1= self.getEloRating(equipo1,equipo2,goles1,goles2,True)
                r2 =self.getEloRating(equipo2,equipo1,goles2,goles1)
                
                self.ratings[equipo1]=r1
                self.ratings[equipo2]=r2
#                 print(self.ratings)
                if(int(data[2])>int(data[4])):
                    self.ganados[int(data[1])-1,int(data[3])-1] +=1
                else:
                    self.ganados[int(data[3])-1,int(data[1])-1] +=1
    def getEloRating(self,equipo1,equipo2,goles1,goles2,esLocal=False,K=60):
        r0_1 = self.ratings[equipo1]
        r0_2 = self.ratings[equipo2]
        
        df1 = int(goles1-goles2)
#         print(df1)
        G =0
        if df1<2:
            G=1
        elif df1 == 2:
            G = 1.5
        else:
            G=(9+df1)/8
#         print("G "+str(G))
#         // W is the result of the game (1 for a win, 0.5 for a draw, and 0 for a loss).
        W=0
        if df1==0:
            W=0.5;
        elif df1>0:
            W=1;

#         print("W "+str(W))

#         // W_e = 1 / (10^(-dr/400) + 1)
        dr=.0;
        if esLocal :
            dr = (r0_1-r0_2+100)*-1;
        else:
            dr = (r0_1-r0_2)*-1;
    
#         print("dr "+str(dr))
        W_e = 1 / (pow(10,(dr/400)) + 1);
#         print("WE "+str(W_e))
        return r0_1+ K*G*(W-W_e);

    def equipos(self):
        return self._equipos
    def partidos(self):
        return self._partidos
    def cmm(self):
        return self._cmm
    def wp(self):
        wp = np.zeros(self._equipos)
        for i in range(0,self._equipos):
            wp[i]=sum(self.ganados[i])/self.totales[i]
        
        return wp
import math


def forward_elimination(A, b, n):
    """
    Calculates the forward part of Gaussian elimination.
    """
    for row in range(0, n-1):
        for i in range(row+1, n):
            factor = A[i,row] / A[row,row]
            for j in range(row, n):
                A[i,j] = A[i,j] - factor * A[row,j]

            b[i] = b[i] - factor * b[row]

        print('A = \n%s and b = %s' % (A,b))
    return A, b

def back_substitution(a, b, n):
    """"
    Does back substitution, returns the Gauss result.
    """
    x = np.zeros((n,1))
    x[n-1] = b[n-1] / a[n-1, n-1]
    for row in range(n-2, -1, -1):
        sums = b[row]
        for j in range(row+1, n):
            sums = sums - a[row,j] * x[j]
        x[row] = sums / a[row,row]
    return x

def gauss(A, b):
    """
    This function performs Gauss elimination without pivoting.
    """
    n = A.shape[0]

    # Check for zero diagonal elements
    if any(np.diag(A)==0):
        raise ZeroDivisionError(('Division by zero will occur; '
                                  'pivoting currently not supported'))
    
    A, b = forward_elimination(A, b, n)
    return back_substitution(A, b, n)

In [128]:
ins = Instancia("tests/test1.in")

['1', '1', '16', '4', '13']
['1', '2', '38', '5', '17']
['1', '2', '0', '6', '1']
['1', '3', '34', '1', '21']
['1', '3', '23', '4', '10']
['1', '4', '31', '1', '6']
['1', '5', '0', '6', '1']
['1', '5', '38', '4', '23']
['1', '6', '1', '2', '0']
['1', '6', '1', '5', '0']
['1', '4', '1', '6', '0']
['1', '3', '1', '1', '0']
['1', '3', '1', '2', '0']


In [48]:
ins.partidos()

13

In [53]:
ins.totales

array([4., 4., 4., 5., 4., 5.])

In [52]:
ins.ganados

array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [2., 1., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 2., 0., 0., 2., 0.]])

In [40]:
for i in range(0, ins.equipos()):
    for j in range(0, ins.equipos()):
        if i==j:
            ins._cmm[i][j]=  ins.totales[i]+ 2
        else:
            ins._cmm[i][j]=  (ins.ganados[i][j]+ ins.ganados[j][i])*-1

In [41]:
ins._cmm

array([[ 5., -0., -1., -2., -0., -0.],
       [-0.,  5., -0., -0., -1., -2.],
       [-1., -0.,  4., -1., -0., -0.],
       [-2., -0., -1.,  7., -1., -1.],
       [-0., -1., -0., -1.,  6., -2.],
       [-0., -2., -0., -1., -2.,  7.]])

In [42]:
b = np.zeros(ins.equipos())

In [47]:
for i in range(0,ins.equipos()):
    ganados =0
    for j in range(0,ins.equipos()):
        ganados += ins.ganados[i][j]
    b[i]= 1+(2*ganados-ins.totales[i])/2

In [49]:
ins.wp()

array([0.25, 0.25, 1.  , 0.4 , 0.25, 0.8 ])

In [129]:
ins.ratings

array([ 46.79087532,  87.94696968, 232.61468536, 146.26090011,
       106.67364898, 161.90997082])

In [79]:
np.ones(7)*2

array([2., 2., 2., 2., 2., 2., 2.])